# Lecture 9: Fast neighbor search methods

## Physics 7810, Spring 2020

## 9.1 - Overview
 
The most time-consuming part of most MC and MD simulations is the energy or force calculation. In a system of $N$ particles with pairwise-additive interaction potentials, this would appear to involve computing $O(N^2)$ pair interactions. However, for systems with *short-range* interactions, it's possible to do substantially better than this by using an interaction cutoff and employing *neighbor lists* (sometimes called *Verlet lists*) or *cell lists*.

The cost of computing short-range interactions is $O(N)$ using cell lists and $\sim O(N^{3/2})$ using neighbor lists, so cell lists are always advantageous for large $N$, but neighbor lists may perform better for intermediate values of $N$, and an *all-pairs* neighbor search may be faster for small $N$. It's also possible to combine neighbor lists and cell lists to obtain a method that outperforms either method separately.

For periodic systems with *long-range* interactions, the Ewald summation method can be used to reduce the computational complexity to $O(N^{3/2})$, and FFT-based variations of the Ewald method such as particle-mesh Ewald (PME) and particle-particle particle-mesh (P$^3$M) can further reduce the computational cost to $O(N \ln N)$. There are also $O(N)$ methods such as the fast multipole (FMM) method.

## 9.2 - Neighbor lists (Verlet lists)
 
In the neighbor list method, we maintain $N$ lists $\cal{L}_i$ of all particles within a distance $r_l$ of each particle $i$, where $r_l$ is somewhat larger than the interaction cutoff $r_c$, as illustrated below:

<img src="images/A&T_Fig_5.3.png" alt="Drawing" style="width: 500px;">

The difference between $r_l$ and $r_c$ is referred to as the neighbor list *skin* $\delta$, i.e., $\delta = r_l - r_c$. The value of $\delta$ can (and should) be adjusted to optimize the performance of the neighbor list algorithm, as discussed below.

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

At the beginning of the simulation, the neighbor lists are constructed, which requires $O(N^2)$ computational effort. For the next few MD timesteps, forces and energies are evaluated using these neighbor lists, which requires $O(N)$ computational cost (we consider MD simulations here for ease of discussion, but neighbor list and cell list methods can also be used for MC simulations, with appropriate modifications).

The current neighbor list can be used until *any particle* has moved far enough that it could potentially have an interaction with another particle that's not in its neighbor list. If the total displacement of particle $i$ since the last update of the neighbor list is $\Delta {\bf r}_i$, then we *update all neighbor lists* as soon as $|\Delta {\bf r}_i| > \delta / 2$ *for any* $i$ (in practice, we use the criterion $|\Delta {\bf r}_i|^2 > (\delta / 2)^2$ to avoid taking square roots).

The overall procedure entails force/energy evaluations with $O(N)$ computational cost and periodic updates of the neighbor lists with $O(N^2)$ cost. Using a larger $\delta$ reduces the frequency of neighbor list updates, but increases the cost of each force/energy evaluation, as there are more pair distances to evaluate at each timestep. The value of $\delta$ can be adjusted to minimize the average CPU time per MD timestep, and the optimal value depend on $N$ (the optimal $\delta$ generally increases with increasing $N$).

Speedup with Verlet neighbor list. Example results are shown for the Lennard-Jones potential, with cutoff $r_c = 2.5$ and various values of skin thickness $\delta$ for the indicated system sizes $N$. The state point is $\rho^\ast = 0.78$, $T^\ast = 0.85$, and the timestep is $\Delta t = 0.005$. The curves show timesteps per CPU second, normalized by the speed for zero skin thickness (when the list is updated every step). The dashed line (also shown in the inset) gives the average number of steps between updates, which is almost independent of system size.  

<img src="images/A&T_Fig_5.4.png" alt="Drawing" style="width: 500px;">

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

What sort of data structures should be used for the neighbor list(s)? Allen and Tildesley use a single one-dimensional array of particle labels for this purpose, but maintaining a separate neighbor list for each particle (e.g., a two-dimensional array) may be simpler.

If your neighbor lists are simple arrays, then you need to make sure that enough memory is allocated to hold the labels of all pairs. The total number of pairs with separation smaller than $r_l$ is of roughly $(4 \pi r_l^3 / 3) \rho (N/2)$, where $\rho$ is the average number density of particles, so the amount of memory needed to store the neighbor lists is of this order.

Looking at my C code, I see that I use linked lists instead of simple arrays to store the neighbor lists, probably to simplify memory management issues, but traversing linked lists generally entails additional computational cost.

For large $N$, the $O(N)$ *cell list* algorithm is faster than the neighbor list method. We'll discuss that method next.

## 9.3 - Cell lists
 
In the cell list method, the simulation box is subdivided into cells with a size $r_\mathrm{cell}$ equal to or slightly greater than the interaction cutoff $r_c$ ($r_\mathrm{cell} \gtrapprox r_c$), as shown below:

<img src="images/A&T_Fig_5.5.png" alt="Drawing" style="width: 750px;">

In the 2D case shown here, there are $n_\mathrm{cell} = L / r_\mathrm{cell}$ cells along each dimension of a square simulation box of size $L$, for a total of $n_\mathrm{cell}^2$ cells. In a 3D cubic simulation box, there are $n_\mathrm{cell}^3$ cells.

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

To find the interacting neighbors of a given particle, we only need to search in the cell containing that particle and in the cells immediately adjacent to that cell.

This entails $O(N)$ computational cost. For the 2D example shown above, there are approximately $\rho r_\mathrm{cell}^2$ particles in each cell, where $\rho$ is the number density, so only $9 N \rho r_\mathrm{cell}^2$ pairs need to considered (or $(9/2) N \rho r_\mathrm{cell}^2$ if we make use of Newton's third law in an MD simulation). In 3D, we need to consider $27 N \rho r_\mathrm{cell}^3$ pairs (or $(27/2) N \rho r_\mathrm{cell}^3$ pairs for MD).

The speedup relative to the $O(N^2)$ all-pairs method is $\approx L^2 / 9 r_\mathrm{cell}^2$ in 2D and $\approx L^3 / 27 r_\mathrm{cell}^3$ in 3D.

This method is equivalent to an all-pairs search for $n_\mathrm{cell} = 3$, so it's only advantageous for $n_\mathrm{cell} \geq 4$.

The search volume in the cell method ($9 r_\mathrm{cell}^2$ in 2D, $27 r_\mathrm{cell}^3$ in 3D) is generally larger than the search volume for the neighbor list method ($\pi r_l^2$ in 2D, $(4 \pi / 3) r_l^3$ in 3D), so the neighbor search algorithm may be faster for small $N$, but will always be slower for large $N$, due to its $\sim O(N^{3/2})$ computational cost.

To reduce the search volume for the cell method, we can reduce the size of the cells and increase the number of neighboring cells that need to be searched. For example, if we take $r_\mathrm{cell} \gtrapprox r_c / 2$, then the search volume becomes $(25/4) r_\mathrm{cell}^2$ in 2D, rather than $9 r_\mathrm{cell}^2$, because we are now searching a $5 \times 5$ domain of cells. In fact, the cells can be chosen to be sufficiently small that each cell either contains either a single particle or no particles.

While this may be advantageous in some cases, the speedup arising from reducing the search volume is at least partially offset by the additional overhead associated with searching more cells.

The cell search algorithm is implemented using linked lists. In this approach, there's a linked list associated with each cell, that's accessed via a `head` array that contains the label of the first particle in the cell list. The number of elements in the `head` array is equal to the number of cells, $N_\mathrm{cells} = n_\mathrm{cell}^d$, where $d$ is the spatial dimensionality.

We also define a `cell` array that specifies which cell a given particle is in. If cells are labeled using a linear index, then `cell` is simply a linear array of length $N$, where $N$ is the number of particles. If a multidimensional cell indexing scheme is used (i.e., indices $i_\mathrm{cell},j_\mathrm{cell}$ in 2D or $i_\mathrm{cell},j_\mathrm{cell},k_\mathrm{cell}$ in 3D) then the `cell` array is an $N \times d$ array.

The cell lists are traversed using `next` and `prev` arrays that contain the labels of the *next* and *previous* particles in the cell list that a given particle belongs to, respectively. The `next` and `prev` arrays are of length $N$.

At the beginning of the simulation, the `head`, `next`, and `prev` arrays are initialized to 'null' values (e.g., $-1$). The cell lists are then populated by assigning particles to cells, which can be done efficiently, e.g., using the floor function:

`i_cell = math.floor((x_i + L / 2) / r_cell)
if i_cell == n_cells:
    i_cell = i_cell - 1`,
    
where $x_i$ is the $x$-coordinate of particle $i$, and $i_\mathrm{cell}$ is the $x$-index of the cell that it's assigned to. The $y$ and $z$ indices of the cell ($j_\mathrm{cell}$ and $k_\mathrm{cell}$) are computed in an analogous way, and (in 3D) we have `cell[i] = [i_cell, j_cell, k_cell]`. Here I've assumed that the cell 'index' is a $d$-dimensional array, but linear indexing of cells can also be used. Note that a multidimensional indexing scheme is convenient for handling periodic boundary conditions, via modular arithmetic applied to the cell indices.

An entry for particle $i$ is then added to the cell list for cell `cell[i]`. If the cell list is empty (`head[cell[i]] == -1`), then we set `head[cell[i]] == i` and `next[i] = prev[i] = -1`. If the cell list isn't empty, then we insert particle $i$ at the head of the list. In Python-like pseudocode, the procedure for adding particle $i$ to a cell list is as follows:

`old_head = head[cell[i]]
if old_head == -1:
    next[i] = prev[i] = -1
    head[cell[i]] == i
else:
    next[i] = old_head
    prev[i] = -1
    prev[old_head] = i
    head[cell[i]] == i`

This process is repeated for every particle $i$ until all particles have been assigned to cells and added to cell lists. Initially, particle indices will be arranged in decreasing order in the cell lists, but this ordering is not preserved over the course of a simulation.

In MD simulations, cell indices are computed and cell lists are updated as needed after each integration timestep. If particle $i$ has moved from one cell into another (i.e., if its cell index has changed), then particle $i$ is removed from the cell list for its old cell and added to the cell list for its new cell. Otherwise, the cell list entries for particle $i$ remain unchanged. Pseudocode for removing particle $i$ from its old cell is as follows:

`if cell[i] != old_cell[i]:
    if next[i] != -1:
        prev[next[i]] = prev[i]
    if prev[i] != -1:
        next[prev[i]] = next[i]
    if head[old_cell[i]] == i:
        head[old_cell[i]] = next[i]`
        
The procedure for adding particle $i$ to the cell list for its new cell is identical to that outlined on the previous slide. In MD simulations, this overall procedure (computing cell indices and updating cell lists if necessary) is carried out for *all particles* after each timestep; in MC simulations, this is done after every trial MC move, just for the particle that was moved.

In MD simulations, the contribution of a particle to the pair interactions (forces, potential energy, virial) is calculated by looping over particles that are further down in the cell list for the cell that the particle belongs to (its 'home' cell) and by looping over particles in neighboring cells. To avoid double counting, only *half* of the neighboring cells are included. Adding the contributions for all particles then yields the total force on each particle as well as the total potential energy and virial.

A somewhat different procedure is required for MC simulations. To compute the change in potential energy in a single-particle move, one must loop over *all other particles in the home cell* for the displaced particle as well as *all particles in all neighboring cells*. To calculate the change in energy, this must be done both before and after the trial move, and the cell lists must be updated to calculate the new energy if the trial move carries the displaced particle out of its home cell (and must be reset if the trial move is rejected).

Finally, it's often useful (but not necessary) to maintain an array of the indices (or index offsets) of neighboring cells to loop over in the cell search procedure.